<link rel="preconnect" href="https://fonts.gstatic.com">
<link href="https://fonts.googleapis.com/css2?family=Fira+Code&display=swap" rel="stylesheet">

### License

<p style="font-family: 'Fira Code', monospace; font-size: 1.2rem">
Copyright 2021 Xilinx, Inc.<br><br>
Licensed under the Apache License, Version 2.0 (the "License");<br>
you may not use this file except in compliance with the License.<br><br>
You may obtain a copy of the License at <a href="http://www.apache.org/licenses/LICENSE-2.0"?>http://www.apache.org/licenses/LICENSE-2.0</a><br><br>
Unless required by applicable law or agreed to in writing, software<br>
distributed under the License is distributed on an "AS IS" BASIS,<br>
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.<br>
See the License for the specific language governing permissions and<br>
limitations under the License.<br>
</p>


# ChipScoPy DDR Reporting Example


<img src="../img/api_overview.png" width="500" align="left">

## Description
This demo shows how to print and report DDR calibration status and report detailed information.


## Requirements
- Local or remote Xilinx Versal board, such as a VCK190
- Xilinx hw_server 2021.1 or greater
- Xilinx cs_server 2021.1 or greater
- Python 3.8 or greater installed
- ChipScoPy 2021.1 or greater installed
- Jupyter notebook support installed - Please do so, using the command `pip install chipscopy[jupyter]`

## 1 - Initialization: Imports and File Paths

After this step,
- Required functions and classes are imported
- Paths to server(s) and files are set correctly

In [1]:
import pprint
import os
import json
from chipscopy import create_session, report_versions
from chipscopy import get_design_files

In [2]:
# Specify locations of the running hw_server and cs_server below.
CS_URL = os.getenv("CS_SERVER_URL", "TCP:localhost:3042")
HW_URL = os.getenv("HW_SERVER_URL", "TCP:localhost:3121")

# The get_design_files() function tries to find the PDI and LTX files. In non-standard
# configurations, you can put the path for PROGRAMMING_FILE and PROBES_FILE below.
design_files = get_design_files("vck190/production/chipscopy_ced")

PDI_FILE = design_files.programming_file
LTX_FILE = design_files.probes_file

print(f"HW_URL: {HW_URL}")
print(f"CS_URL: {CS_URL}")
print(f"PROGRAMMING_FILE: {PDI_FILE}")
print(f"PROBES_FILE:{LTX_FILE}")

HW_URL: TCP:localhost:3121
CS_URL: TCP:localhost:3042
PROGRAMMING_FILE: C:\wrk\chipscopy\chipscopy\examples\designs\vck190\production\chipscopy_ced\chipscopy_ex_wrapper.pdi
PROBES_FILE:C:\wrk\chipscopy\chipscopy\examples\designs\vck190\production\chipscopy_ced\chipscopy_ex_wrapper.ltx


## 2 - Create a session and connect to the hw_server and cs_server

The session is a container that keeps track of devices and debug cores.
After this step,
- Session is initialized and connected to server(s)
- Versions are detected and reported to stdout

In [3]:
# Start of the connection
session = create_session(cs_server_url=CS_URL, hw_server_url=HW_URL)
report_versions(session)

## 3 - Program the device with the example design

After this step,
- Device is programmed with the example programming file

In [4]:
# Typical case - one device on the board - get it.
versal_device = session.devices.get()
versal_device.program(PDI_FILE)

Output()

## 4 - Discover Debug Cores

Debug core discovery initializes the chipscope server debug cores. This brings debug cores in the chipscope server online.

After this step,

- The cs_server is initialized and ready for use

In [5]:
versal_device.discover_and_setup_cores()
print(f"Debug cores setup and ready for use.")

Debug cores setup and ready for use.


## 5 - Show enabled DDRs in the device. Pick one to use

In [6]:
ddr_list = versal_device.ddrs
for ddr in ddr_list:
    print(ddr, "  Enabled:", ddr.is_enabled)

# Grab the first enabled DDR
for ddr in ddr_list:
    if ddr.is_enabled:
        print("Using Enabled DDR: ", ddr)
        break

ddr_0   Enabled: True
ddr_1   Enabled: False
ddr_2   Enabled: False
ddr_3   Enabled: False
Using Enabled DDR:  ddr_0


## 6 - Getting the Calibration Status

There are several methods available to collect memory calibration information.

### Method 1 - Calibration PASS/FAIL status

In [7]:
# Method 1 - Use Status string base API directly
print(ddr, "calibration status:", ddr.get_cal_status())

ddr_0 calibration status: PASS


### Method 2 - Calibration from the status property group

In [8]:
# Use Property Group to get dictionary base of results
props = ddr.ddr_node.get_property_group(["status"])
print(pprint.pformat(props, indent=2))

{ 'cal_error_msg': 'None',
  'cal_message': 'No errors detected during calibration.',
  'cal_status': 'PASS',
  'dqs_status': 'Running',
  'health_status': 'GOOD',
  'track_enable': 'DQS Gate Tracking, UB ISR Tracking, Post-Cal Status, DDRMC '
                  'ISR Tracking'}


### Method 3 - Detailed calibration status for each stage

In [9]:
# Use get Cal Stages API directly to also get dictionary results
props = ddr.get_cal_stages()
print(pprint.pformat(sorted(props.items()), indent=2))

[ ('cal_stage.01_F0_PHY_BISC', 'Pass'),
  ('cal_stage.02_F0_MEM_INIT', 'Pass'),
  ('cal_stage.03_F0_DQS_GATE_CAL', 'Pass'),
  ('cal_stage.04_F0_WRITE_LEVELING', 'Pass'),
  ('cal_stage.05_F0_READ_DQ_CAL', 'Pass'),
  ('cal_stage.06_F0_WRITE_DQ_DBI_CAL', 'Pass'),
  ('cal_stage.07_F0_WRITE_LATENCY_CAL', 'Pass'),
  ('cal_stage.08_F0_READ_DQ_DBI_CAL_COMPLEX', 'Pass'),
  ('cal_stage.09_F0_WRITE_DQ_DBI_CAL_COMPLEX', 'Pass'),
  ('cal_stage.10_EN_VT_TRACK', 'Pass'),
  ('cal_stage.11_READ_DQS_TRACK', 'Pass'),
  ('cal_stage.12_CAL_DONE', 'Pass')]


## 7 - Generate Full DDRMC Report 

The report() API call creates a full DDRMC status report to stdout or a file. This report includes memory configuration, margin analysis, calibration, and health status information.

In [10]:
# Use a single report command to get all latest essential
# Status and decoded data collected as it presents
ddr.report()
# Specify True to argument 1, and name/path to argument 2
# to get the report output generated and saved to a file
ddr.report(True, "test_out.txt")
print("Report Done.\n")

-------------------

 DDRMC Status 

-------------------

Calibration Status:   PASS 

Overall Health:   GOOD 

Message:   No errors detected during calibration. 


-------------------

 Status Registers

-------------------

DDRMC ISR Table

   ddrmc_isr_ce0_ecc0 :   0
   ddrmc_isr_ce0_ecc1 :   0
   ddrmc_isr_ce1_ecc0 :   0
   ddrmc_isr_ce1_ecc1 :   0
   ddrmc_isr_ch0_data_par :   0
   ddrmc_isr_ch1_data_par :   0
   ddrmc_isr_dc_cmd0_fatal :   0
   ddrmc_isr_dc_cmd1_fatal :   0
   ddrmc_isr_dram_parity0 :   0
   ddrmc_isr_dram_parity1 :   0
   ddrmc_isr_dram_parity_fatal_0 :   0
   ddrmc_isr_dram_parity_fatal_1 :   0
   ddrmc_isr_na_cmd :   0
   ddrmc_isr_na_cmd0 :   0
   ddrmc_isr_na_cmd1 :   0
   ddrmc_isr_na_cmd_fatal :   0
   ddrmc_isr_nsu_0 :   0
   ddrmc_isr_nsu_1 :   0
   ddrmc_isr_nsu_2 :   0
   ddrmc_isr_nsu_3 :   0
   ddrmc_isr_nsu_fatal0 :   0
   ddrmc_isr_nsu_fatal1 :   0
   ddrmc_isr_nsu_fatal2 :   0
   ddrmc_isr_nsu_fatal3 :   0
   ddrmc_isr_uc0_ecc0 :   0
   ddrmc_isr_

Byte  1  Nibble 1  -   Left Margin:   116  ( 57 )  Center Point:   170  ( 84 )  Right Margin:   116  ( 57 )
Byte  2  Nibble 0  -   Left Margin:   119  ( 58 )  Center Point:   167  ( 81 )  Right Margin:   119  ( 58 )
Byte  2  Nibble 1  -   Left Margin:   109  ( 55 )  Center Point:   158  ( 80 )  Right Margin:   111  ( 56 )
Byte  3  Nibble 0  -   Left Margin:   123  ( 59 )  Center Point:   162  ( 78 )  Right Margin:   125  ( 60 )
Byte  3  Nibble 1  -   Left Margin:   125  ( 60 )  Center Point:   169  ( 81 )  Right Margin:   125  ( 60 )
Byte  4  Nibble 0  -   Left Margin:   116  ( 54 )  Center Point:   165  ( 77 )  Right Margin:   118  ( 55 )
Byte  4  Nibble 1  -   Left Margin:   120  ( 57 )  Center Point:   175  ( 83 )  Right Margin:   122  ( 58 )
Byte  5  Nibble 0  -   Left Margin:   124  ( 61 )  Center Point:   164  ( 81 )  Right Margin:   126  ( 62 )
Byte  5  Nibble 1  -   Left Margin:   129  ( 62 )  Center Point:   175  ( 84 )  Right Margin:   129  ( 62 )
Byte  6  Nibble 0  -   Left 

## 8 - Dump the complete set of internal properties as json

This demonstrates how to get a Python dictionary of all the low level DDR properties. These can be converted to JSON easily for export to other tools.

In [11]:
props = ddr.ddr_node.get_property_group([])
json_props=json.dumps(props, indent=4)
print(json_props)

{
    "f0_wrdqdbi_stg3_dq_odly_bit41": 46,
    "f0_wrprbs_right_margin_byte4": 0,
    "vttrack_rldlyqtr_min_nibble08": 72,
    "f1_wrdqdbi_stg2_dq_odly_bit38": 0,
    "f0_mr4_17_9": 0,
    "f1_wrdqdbi_stg3_dq_odly_bit57": 0,
    "f1_xpll_divclk_ht": 1,
    "f0_rdcmplx_nqtr_right_short_fcrse_nibble15": 160,
    "wrvref_vref_final_rank0_byte5": 0,
    "f0_wrlvl_fine_final_rank0_byte3": 74,
    "f1_rddq_idelay_final_bit19": 0,
    "f1_wrdqdbi_deskew_dq_odly_bit00": 0,
    "cal_seq_status_20": 287,
    "f1_wrdqdbi_left_edge_dq_bit56": 0,
    "rdvref_right_vref_upp_bound_nibble06": 0,
    "f0_rddq_nqtr_left_nibble11": 21,
    "f0_rd_simp_rise_rm_nib_02": [
        72,
        150
    ],
    "f1_wrdqdbi_deskew_dq_odly_fcrse_bit49": 0,
    "wrvref_right_vref_final_rank0_byte4": 0,
    "f1_wrprbs_right_margin_byte3": 0,
    "f0_wrdqdbi_odly_dbi_final_byte0": 61,
    "f1_wrdqdbi_stg3_dq_odly_bit22": 0,
    "wrmargin_left_byte6": 0,
    "f0_calbisc_rl_dly_nqtr_nibble03": 77,
    "f0_wrdqdbi_stg2

}
